# Histogram Equalization Examples

Comparing the Python and Julia implementations of the CLAHE code.

In [ ]:
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("PyCall", "Images", "DataFrames")

Standard imports:

In [ ]:
using IceFloeTracker, Images, DataFrames
import IceFloeTracker.Filtering: sk_exposure, adjust_histogram, ContrastLimitedAdaptiveHistogramEqualization
using PyCall
@pyimport numpy as np

Use the standard dataset:

In [ ]:
dataset = Watkins2026Dataset()

## Histogram Equalization
We call `adjust_histogram` with `Equalization` implemented in Julia and `sk_exposure.equalize_hist` implemented in Python, and compare the results.

In [ ]:
case = dataset[21]
casered = red.(modis_truecolor(case))
img_eq_jl = adjust_histogram(casered, Equalization(; nbins=128))
img_eq_py = sk_exposure.equalize_hist(np.array(casered); nbins=128)
difference = float.(img_eq_jl) .- float.(img_eq_py)
display(name(case))
display(
    hcat(Gray.(casered),
        Gray.(img_eq_py),
        Gray.(img_eq_jl),
        Gray.(0.5 .+ difference .* 5))
)

To compare many cases, we need a function. 

In general, we want to compare 
- the status quo Python result to the original, 
- the original to the new implementation, 
- and the difference between the two results.

We use the `compare` function to calculate the following images shown from left to right in the example below:
- Python output
- difference Python vs original,
- original,
- difference Julia vs original,
- Julia output
- difference Julia vs Python.

Differences are shown as offsets from a half-gray tone, so flat gray means no difference. 

In [ ]:
function compare(
    img;
    JlAlgorithm=Equalization(; nbins=128),
    py_function=sk_exposure.equalize_hist,
    py_kwargs=Dict(:nbins => 128),
    difference_offset=0.5,
    difference_factor=1.0
)

    img_eq_jl = adjust_histogram(img, JlAlgorithm)
    img_eq_py = py_function(np.array(img); py_kwargs...)
    
    im_diff_py = float.(img) .- float.(img_eq_py)
    im_diff_jl = float.(img) .- float.(img_eq_jl)
    jl_diff_py = float.(img_eq_jl) .- float.(img_eq_py)

    result = (
        py=Gray.(img_eq_py),
        im_diff_py=Gray.(0.5 .+ ((im_diff_py) * difference_factor)),
        im=Gray.(img),
        im_diff_jl=Gray.(0.5 .+ ((im_diff_jl) * difference_factor)),
        jl=Gray.(img_eq_jl),
        jl_diff_py=Gray.(0.5 .+ ((jl_diff_py) * difference_factor)),
    )
    return result

end

function compare(dataset::Dataset; kwargs...)
    results = []
    for case in dataset
        @info "Processing case: $(name(case))"
        img = modis_truecolor(case)
        channel1 = red.(img)
        img_results = compare(channel1; kwargs...)

        img_eq_jl = img_results.jl
        img_eq_py = img_results.py

        result = (name=name(case),
            ssim_original_py=assess_ssim(channel1, img_eq_py),
            ssim_original_jl=assess_ssim(channel1, img_eq_jl),
            ssim_jl_py=assess_ssim(img_eq_jl, img_eq_py),
            img_results...)
        push!(results, result)
    end
    results_df = DataFrame(results)
    return results_df
end

JlAlgorithmHE = Equalization(; nbins=128)
py_function_he = sk_exposure.equalize_hist
py_kwargs_he = Dict(:nbins => 128)

display(hcat(compare(red.(modis_truecolor(dataset[174*2])); JlAlgorithm=JlAlgorithmHE, py_function=py_function_he, py_kwargs=py_kwargs_he)...))

We next look at the full validation dataset. 
We want to find cases where the Julia output is maximally different from the Python output. 
To find cases which are perceptually different, we use the structural similarity index measure (SSIM),
where 1 is identical and 0 is completely different.

In [ ]:
results_he = compare(dataset; JlAlgorithm=JlAlgorithmHE, py_function=py_function_he, py_kwargs=py_kwargs_he);

The "worst" cases are shown below.

- The Julia outputs in very low contrast, open water regions are perhaps better than the python, retaining the average very dark color more consistently. 
- In high-contrast parts of the image the performance seems similar. 

In [ ]:
for row in eachrow(sort(results_he, :ssim_jl_py))[1:20]
    display("$(row.name), ssim=$(round(row.ssim_jl_py, digits=2))")
    display(hcat(row.im, row.py, row.jl, row.jl_diff_py))
end

## Contrast Limited Adaptive Histogram Equalization
We call `adjust_histogram` with `ContrastLimitedAdaptiveHistogramEqualization` implemented in Julia and `sk_exposure.equalize_adapthist` implemented in Python, and compare the results.

In [ ]:
case = dataset[21]
casered = red.(modis_truecolor(case))
img_eq_jl = adjust_histogram(casered, ContrastLimitedAdaptiveHistogramEqualization(; clip=2.0))
img_eq_py = sk_exposure.equalize_adapthist(np.array(casered); clip_limit=0.01)
difference = img_eq_jl .- img_eq_py
display(name(case))
display(
    hcat(Gray.(casered), 
    Gray.(img_eq_py ), 
    Gray.(img_eq_jl ), 
    Gray.(0.5 .+ difference .* 5))
)

Using the `compare` utility:

In [ ]:
JlAlgorithmCLAHE = ContrastLimitedAdaptiveHistogramEqualization(; clip=2.0, nbins=256)
py_function_clahe = sk_exposure.equalize_adapthist
py_kwargs_clahe = Dict(:clip_limit => 0.01)

display(hcat(compare(casered; JlAlgorithm=JlAlgorithmCLAHE, py_function=py_function_clahe, py_kwargs=py_kwargs_clahe)...));

We next look at the full validation dataset. 
We want to find cases where the Julia output is maximally different from the Python output. 
To find cases which are perceptually different, we use the structural similarity index measure (SSIM),
where 1 is identical and 0 is completely different.

In [ ]:
results_clahe = compare(dataset; JlAlgorithm=JlAlgorithmCLAHE, py_function=py_function_clahe, py_kwargs=py_kwargs_clahe);

The "worst" cases are shown below.

The image order is:
- Original
- Python result,
- Julia result,
- Python vs Julia difference, scaled as differences around half-gray.

The results from Julia are very similar to those from Python.

In [ ]:
for row in eachrow(sort(results_clahe, :ssim_jl_py))[1:20]
    display("$(row.name), ssim=$(round(row.ssim_jl_py, digits=2))")
    display(hcat(row.im, row.py, row.jl, row.jl_diff_py))
end